In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import transformers

4.54.0


In [ ]:
import torch
# from musicbert_hf import MusicBert, MusicBertConfig

In [ ]:
!pip install pretty_midi mido

In [ ]:
!pip install miditok pretty_midi

In [ ]:
import os
from google.colab import drive
import librosa
import soundfile as sf
import numpy as np
from pathlib import Path
from miditok import REMI
from miditok.constants import CHORD_MAPS
import pretty_midi
import json
from tqdm import tqdm
import pickle
from miditok import TokenizerConfig
import pretty_midi

In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("imsparsh/musicnet-dataset")

print("Contents of downloaded path:")
print(os.listdir(path))

for item in os.listdir(path):
    item_path = os.path.join(path, item)
    if os.path.isdir(item_path):
        print(f"\nContents of {item}:")
        print(os.listdir(item_path))

100%|██████████| 21.5G/21.5G [17:01<00:00, 22.6MB/s]

Extracting files...


Contents of downloaded path:
['musicnet_metadata.csv', 'musicnet.npz', 'musicnet', 'musicnet_midis']

Contents of musicnet:
['musicnet']

Contents of musicnet_midis:
['musicnet_midis']


In [ ]:
# Download latest version
OUTPUT_MIDI_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_midi_segments"
METADATA_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_midi_metadata"

os.makedirs(OUTPUT_MIDI_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

def segment_midi(midi_file_path, segment_length=10.0, stride=5.0):
    # 切MIDI，10s一段，5秒overlap
    segments = []
    metadata = []
    # 存分段和描述信息

    midi = pretty_midi.PrettyMIDI(midi_file_path)
    total_duration = midi.get_end_time()

    start = 0.0
    segment_id = 0

    while start < total_duration:
        end = start + segment_length

        # Create segment by copying notes within time range
        segment_midi = pretty_midi.PrettyMIDI()
        for instrument in midi.instruments:
            new_instrument = pretty_midi.Instrument(
                program=instrument.program,
                is_drum=instrument.is_drum
            )
            for note in instrument.notes:
                if note.start < end and note.end > start:
                    # Adjust note timing relative to segment start
                    new_note = pretty_midi.Note(
                        velocity=note.velocity,
                        pitch=note.pitch,
                        start=max(0, note.start - start),
                        end=min(segment_length, note.end - start)
                    )
                    new_instrument.notes.append(new_note)
            if new_instrument.notes:
                segment_midi.instruments.append(new_instrument)

        is_padded = end > total_duration
        # 处理MIDI结尾分段超过整首歌长度

        segments.append(segment_midi)
        # 加入分段
        metadata.append({
            "segment_id": segment_id,
            "start_time": float(start),
            "end_time": float(min(end, total_duration)),
            "duration": segment_length,
            "is_padded": is_padded,
            "overlap_previous": start > 0,
            "overlap_next": (end < total_duration),
        })
        # 加入分段信息
        start += stride
        segment_id += 1

    return segments, metadata

# Get composer directories
composer_dirs = os.listdir(os.path.join(path, "musicnet_midis", "musicnet_midis"))

all_segments = []
all_metadata = []
processed_count = 0
error_count = 0

# Loop through all composer directories
for composer in composer_dirs:
    composer_path = os.path.join(path, "musicnet_midis", "musicnet_midis", composer)
    midi_files = os.listdir(composer_path)

    for midi_file in midi_files:
        if midi_file.endswith('.mid'):
            file_path = os.path.join(composer_path, midi_file)
            try:
                segments, metadata = segment_midi(file_path)
                processed_count += 1

                for seg_obj, meta in zip(segments, metadata):
                    # build a unique filename per segment
                    base = os.path.splitext(midi_file)[0]
                    seg_name = f"{base}_seg{meta['segment_id']}.mid"
                    out_path = os.path.join(OUTPUT_MIDI_DIR, seg_name)

                    # dump the PrettyMIDI segment to disk
                    seg_obj.write(out_path)

                    # ── Clean & save per-file metadata JSON ───────────────────
                clean_meta = []
                for m in metadata:
                    clean_meta.append({
                        "segment_id":       int(  m["segment_id"]       ),
                        "start_time":       float(m["start_time"]       ),
                        "end_time":         float(m["end_time"]         ),
                        "duration":         float(m["duration"]         ),
                        "is_padded":        bool( m["is_padded"]        ),
                        "overlap_previous": bool( m["overlap_previous"] ),
                        "overlap_next":     bool( m["overlap_next"]     ),
                    })

                meta_path = os.path.join(METADATA_DIR, f"{base}_metadata.json")
                with open(meta_path, 'w') as mf:
                    json.dump(clean_meta, mf, indent=2)


                if processed_count % 50 == 0:  # Progress update
                    print(f"Processed {processed_count} files...")

            except Exception as e:
                print(f"Error processing {composer}/{midi_file}: {str(e)}")
                error_count += 1

print(f"Total processed: {processed_count} MIDI files")
print(f"Total errors: {error_count} MIDI files")
num_files = len([f for f in os.listdir(OUTPUT_MIDI_DIR) if f.endswith('.mid')])
print(f"Total MIDI segments written: {num_files}")

KeyboardInterrupt: 

In [ ]:
!pip install miditoolkit

In [ ]:
from miditoolkit import MidiFile

In [ ]:
from miditoolkit import MidiFile
from pathlib import Path
import json
import pickle
import os

from miditok import REMI
from miditok import TokenizerConfig

# Create the directories

TOKENIZER_CONFIG = TokenizerConfig(
    beat_res={(0, 4): 8, (4, 12): 4},
    num_velocities=32,
    additional_tokens={
        "Chord": True,
        "Rest": True,
        "Tempo": True,
        "Program": False,
        "TimeSignature": False
    }
)

tokenizer = REMI(TOKENIZER_CONFIG)

# Update the MIDI input directory path for Google Colab mounted Drive
OUTPUT_MIDI_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_midi_segments"
# OUTPUT_MIDI_DIR = "/content/drive/MyDrive/MuseGuard/musicnet_midi_segments"
TOKENIZED_DIR = "/content/drive/MyDrive/MuseGuard/tokenized_segments"
EVENTS_DIR = "/content/drive/MyDrive/MuseGuard/event_objects"

os.makedirs(TOKENIZED_DIR, exist_ok=True)
os.makedirs(EVENTS_DIR, exist_ok=True)

def tokenize_midi_segments():
    """
    Tokenize all the .mid segments you wrote to OUTPUT_MIDI_DIR,
    saving tokens and events to TOKENIZED_DIR and EVENTS_DIR.
    """
    midi_paths = sorted(Path(OUTPUT_MIDI_DIR).glob("*.mid"))
    print(f"→ Tokenizing {len(midi_paths)} MIDI segments from {OUTPUT_MIDI_DIR}")

    tokenized_data = []

    for midi_path in midi_paths:
        midi_fname = midi_path.name
        try:
            # 1) Quick emptiness check
            midi_obj = MidiFile(str(midi_path))
            if not any(inst.notes for inst in midi_obj.instruments):
                print(f"  Skipping empty: {midi_fname}")
                continue

            # 2) Tokenize by passing the path
            seqs = tokenizer(str(midi_path))
            if not isinstance(seqs, list):
                seqs = [seqs]

            # 3) Flatten all TokSequence → tokens + events
            tokens = [tok for seq in seqs for tok in seq.tokens]
            events = [ev  for seq in seqs for ev  in seq.events]

            # 4) Record in-memory
            tokenized_data.append({
                'filename':    midi_fname,
                'token_count': len(tokens),
                'event_count': len(events),
                'tokens':      tokens,
                'events':      events
            })

            # 5) Dump per-segment token JSON
            tok_file = TOKENIZED_DIR + f"/tokens_{midi_fname.replace('.mid','.json')}"
            with open(tok_file, 'w') as f:
                json.dump({
                    'filename':    midi_fname,
                    'tokens':      tokens,
                    'token_count': len(tokens)
                }, f, indent=2)


            # 6) Dump per-segment events pickle
            ev_file = EVENTS_DIR + f"/events_{midi_fname.replace('.mid','.pkl')}"
            with open(ev_file, 'wb') as f:
                pickle.dump({
                    'filename':    midi_fname,
                    'events':      events,
                    'event_count': len(events)
                }, f)

            print(f"  ✅ {midi_fname}: {len(tokens)} tokens, {len(events)} events")

        except Exception as e:
            print(f"  ❌ Error {midi_fname}: {e}")
            continue

    # 7) Save the full summary JSON
    json_safe_data = []
    for item in tokenized_data:
        # Convert events to dictionaries
        events_as_dicts = []
        for event in item['events']:
            event_dict = {
                'type': event.type,
                'value': event.value,
                'time': event.time if hasattr(event, 'time') else None,
                # Add other attributes if your Event class has them
            }
            events_as_dicts.append(event_dict)

        # Create JSON-safe version
        json_safe_item = {
            'filename': item['filename'],
            'token_count': item['token_count'],
            'event_count': item['event_count'],
            'tokens': item['tokens'],
            'events': events_as_dicts  # Now JSON serializable
        }
        json_safe_data.append(json_safe_item)

    with open(f"{TOKENIZED_DIR}/complete_tokenized_dataset.json", 'w') as f:
        json.dump({
            'total_segments': len(json_safe_data),
            'data': json_safe_data
        }, f, indent=2)


    # 8) Persist the tokenizer config for reuse
    with open(f"{TOKENIZED_DIR}/tokenizer_config.pkl", 'wb') as f:
        pickle.dump(TOKENIZER_CONFIG, f)

    print(f"\n→ Tokenization complete: {len(tokenized_data)} segments processed")
    return tokenized_data

# And run it:
tokenized_segments = tokenize_midi_segments()

→ Tokenizing 22031 MIDI segments from /content/drive/MyDrive/MuseGuard/musicnet_midi_segments
  ✅ 1727_schubert_op114_2_seg0.mid: 303 tokens, 303 events
  ✅ 1727_schubert_op114_2_seg1.mid: 406 tokens, 406 events
  ✅ 1727_schubert_op114_2_seg10.mid: 505 tokens, 505 events
  ✅ 1727_schubert_op114_2_seg11.mid: 553 tokens, 553 events
  ✅ 1727_schubert_op114_2_seg12.mid: 657 tokens, 657 events
  ✅ 1727_schubert_op114_2_seg13.mid: 668 tokens, 668 events
  ✅ 1727_schubert_op114_2_seg14.mid: 636 tokens, 636 events
  ✅ 1727_schubert_op114_2_seg15.mid: 554 tokens, 554 events
  ✅ 1727_schubert_op114_2_seg16.mid: 521 tokens, 521 events
  ✅ 1727_schubert_op114_2_seg17.mid: 554 tokens, 554 events
  ✅ 1727_schubert_op114_2_seg18.mid: 550 tokens, 550 events
  ✅ 1727_schubert_op114_2_seg19.mid: 531 tokens, 531 events
  ✅ 1727_schubert_op114_2_seg2.mid: 362 tokens, 362 events
  ✅ 1727_schubert_op114_2_seg20.mid: 545 tokens, 545 events
  ✅ 1727_schubert_op114_2_seg21.mid: 530 tokens, 530 events
  ✅ 1727_

KeyboardInterrupt: 

In [ ]:
!pip install sentence-transformers

In [ ]:
import os
import json
import pickle
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [ ]:
# Individual Feature Extractor - One token file → One feature file
import os
import json
import pickle
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import glob
from pathlib import Path
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuration
model_name = "BAAI/bge-base-en-v1.5"
TOKENIZED_DIR = "/content/drive/MyDrive/MuseGuard/tokenized_segments"
FEATURES_DIR = "/content/drive/MyDrive/MuseGuard/extracted_features"
TOKENIZER_PATH = "/content/drive/MyDrive/MuseGuard/remi_tokenizer.pkl"

# Create directories
os.makedirs(FEATURES_DIR, exist_ok=True)

# Load model and tokenizer once
print("Loading BGE model...")
feature_model = SentenceTransformer(model_name)

print("Loading MIDI tokenizer...")
try:
    with open(TOKENIZER_PATH, 'rb') as f:
        midi_tokenizer = pickle.load(f)
    print("✅ Tokenizer loaded successfully")
except Exception as e:
    print(f"❌ Error loading tokenizer: {e}")
    exit(1)

def tokens_to_text(tokens, tokenizer):
    """Convert MIDI tokens to text representation"""
    if not hasattr(tokenizer, 'vocab'):
        if hasattr(tokenizer, 'id_to_token'):
            token_names = []
            for token_id in tokens:
                try:
                    token_name = tokenizer.id_to_token(token_id)
                    token_names.append(token_name if token_name else f"UNK_{token_id}")
                except:
                    token_names.append(f"UNK_{token_id}")
        else:
            token_names = [str(token_id) for token_id in tokens]
    else:
        token_names = []
        for token_id in tokens:
            for token_name, tid in tokenizer.vocab.items():
                if tid == token_id:
                    token_names.append(token_name)
                    break
            else:
                token_names.append(f"UNK_{token_id}")

    return " ".join(token_names)

def process_single_token_file(token_filepath):
    """
    Process a single token file and create corresponding feature file
    Returns: (success, input_file, output_file, feature_shape, processing_time)
    """
    start_time = time.time()

    try:
        # Generate output filename
        token_filename = os.path.basename(token_filepath)
        feature_filename = token_filename.replace('.json', '_features.npy')
        feature_filepath = os.path.join(FEATURES_DIR, feature_filename)

        # Skip if feature file already exists
        if os.path.exists(feature_filepath):
            file_size = os.path.getsize(feature_filepath)
            processing_time = time.time() - start_time
            return (True, token_filename, feature_filename, "already_exists", processing_time)

        # Load token file
        with open(token_filepath, 'r') as f:
            token_data = json.load(f)

        # Extract tokens from different possible structures
        if isinstance(token_data, dict):
            if 'tokens' in token_data:
                tokens = token_data['tokens']
                filename = token_data.get('filename', token_filename.replace('.json', '.mid'))
            elif 'data' in token_data and len(token_data['data']) > 0:
                # Take first segment if multiple
                tokens = token_data['data'][0]['tokens']
                filename = token_data['data'][0].get('filename', token_filename.replace('.json', '.mid'))
            else:
                raise ValueError("No tokens found in file structure")
        elif isinstance(token_data, list) and len(token_data) > 0:
            tokens = token_data
            filename = token_filename.replace('.json', '.mid')
        else:
            raise ValueError("Unrecognized token file structure")

        if not tokens:
            raise ValueError("Empty token list")

        # Convert tokens to text
        text_repr = tokens_to_text(tokens, midi_tokenizer)

        # Truncate if too long
        max_length = 512
        if len(text_repr.split()) > max_length:
            text_repr = " ".join(text_repr.split()[:max_length])

        # Extract features
        features = feature_model.encode([text_repr], convert_to_tensor=True)
        features = features.cpu().numpy()[0]

        # Normalize features
        features = features / np.linalg.norm(features)

        # Save feature file
        np.save(feature_filepath, features)

        processing_time = time.time() - start_time
        return (True, token_filename, feature_filename, features.shape, processing_time)

    except Exception as e:
        processing_time = time.time() - start_time
        return (False, os.path.basename(token_filepath), f"ERROR: {str(e)}", None, processing_time)

def process_all_token_files_parallel(max_workers=4):
    """
    Process all token files in parallel to create individual feature files
    """
    # Find all token files
    token_files = glob.glob(os.path.join(TOKENIZED_DIR, "*.json"))
    token_files = [f for f in token_files if not f.endswith('.backup')]

    # Filter out combined dataset files
    token_files = [f for f in token_files if 'complete_tokenized_dataset' not in os.path.basename(f)
                   and 'fast_combined_dataset' not in os.path.basename(f)
                   and 'optimized_complete_dataset' not in os.path.basename(f)]

    total_files = len(token_files)
    print(f"Processing {total_files} tokenized segments...")

    if total_files == 0:
        print("No token files found!")
        return

    start_time = time.time()
    successful = 0
    failed = 0

    # Process files in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_file = {executor.submit(process_single_token_file, filepath): filepath
                         for filepath in token_files}

        # Process results as they complete
        for i, future in enumerate(as_completed(future_to_file)):
            success, input_file, output_info, shape, proc_time = future.result()

            if success and shape != "already_exists":
                successful += 1
            elif not success:
                failed += 1
                print(f"Error processing {input_file}: {output_info}")

            # Progress update every 10 files (like original)
            if (i + 1) % 10 == 0:
                print(f"Processed {i+1}/{total_files} segments")

    print(f"\nFeature extraction complete!")
    print(f"Total features extracted: {successful}")
    if failed > 0:
        print(f"Failed files: {failed}")
    print(f"Features saved to: {FEATURES_DIR}")

def verify_feature_files():
    """
    Verify that feature files were created correctly
    """
    print("\n=== Feature Statistics ===")

    feature_files = glob.glob(os.path.join(FEATURES_DIR, "*_features.npy"))
    if not feature_files:
        print("No features available - empty feature matrix")
        return

    print(f"Total features extracted: {len(feature_files)}")

    # Load a sample to check dimensions
    try:
        sample_features = np.load(feature_files[0])
        print(f"Feature dimension: {sample_features.shape[0]}")
        print(f"Number of segments: {len(feature_files)}")

        # Sample statistics from first few files
        all_sample_features = []
        for feature_file in feature_files[:10]:  # Sample first 10
            features = np.load(feature_file)
            all_sample_features.append(features)

        if all_sample_features:
            sample_matrix = np.array(all_sample_features)
            print(f"Mean feature value: {np.mean(sample_matrix):.4f}")
            print(f"Std feature value: {np.std(sample_matrix):.4f}")
            print(f"Min feature value: {np.min(sample_matrix):.4f}")
            print(f"Max feature value: {np.max(sample_matrix):.4f}")

    except Exception as e:
        print(f"Error loading sample features: {e}")

# Run the processing
if __name__ == "__main__":
    process_all_token_files_parallel(max_workers=4)
    verify_feature_files()

    print(f"\n=== Pipeline Summary ===")
    print(f"Extracted features: {FEATURES_DIR}")
    print(f"\nFeature extraction pipeline completed successfully!")

Loading BGE model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading MIDI tokenizer...
✅ Tokenizer loaded successfully
Processing 22010 tokenized segments...
Processed 10/22010 segments
Processed 20/22010 segments
Processed 30/22010 segments
Processed 40/22010 segments
Processed 50/22010 segments
Processed 60/22010 segments
Processed 70/22010 segments
Processed 80/22010 segments
Processed 90/22010 segments
Processed 100/22010 segments
Processed 110/22010 segments
Processed 120/22010 segments
Processed 130/22010 segments
Processed 140/22010 segments
Processed 150/22010 segments
Processed 160/22010 segments
Processed 170/22010 segments
Processed 180/22010 segments
Processed 190/22010 segments
Processed 200/22010 segments
Processed 210/22010 segments
Processed 220/22010 segments
Processed 230/22010 segments
Processed 240/22010 segments
Processed 250/22010 segments
Processed 260/22010 segments
Processed 270/22010 segments
Processed 280/22010 segments
Processed 290/22010 segments
Processed 300/22010 segments
Processed 310/22010 segments
Processed 320

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Processed 2450/22010 segments
Processed 2460/22010 segments
Processed 2470/22010 segments
Processed 2480/22010 segments
Processed 2490/22010 segments
Processed 2500/22010 segments
Processed 2510/22010 segments
Processed 2520/22010 segments
Processed 2530/22010 segments
Processed 2540/22010 segments
Processed 2550/22010 segments
Processed 2560/22010 segments
Processed 2570/22010 segments
Processed 2580/22010 segments
Processed 2590/22010 segments
Processed 2600/22010 segments
Processed 2610/22010 segments
Processed 2620/22010 segments
Processed 2630/22010 segments
Processed 2640/22010 segments
Processed 2650/22010 segments
Processed 2660/22010 segments
Processed 2670/22010 segments
Processed 2680/22010 segments
Processed 2690/22010 segments
Processed 2700/22010 segments
Processed 2710/22010 segments
Processed 2720/22010 segments
Processed 2730/22010 segments
Processed 2740/22010 segments
Processed 2750/22010 segments
Processed 2760/22010 segments
Processed 2770/22010 segments
Processed 